# AymaraAI Multiturn Example

This notebook demonstrates a multiturn evaluation workflow with AymaraSDK using thread-based prompt chaining:

- Creating an eval with AymaraSDK
- Fetching eval prompts
- For each prompt, simulating a 3-turn conversation using `client.evals.runs.continue_run` and `continue_thread`
- Creating an eval run with the multiturn conversations
- Generating and displaying a report

## Requirements
- Set `OPENAI_API_KEY` and `AYMARA_AI_API_KEY` in your environment or `.env` file.
- Install dependencies: `pip install openai aymara-ai dotenv`

In [ ]:
# Environment and imports
import os
from typing import List

import openai
import pandas as pd
from dotenv import load_dotenv

from aymara_ai import AymaraAI
from aymara_ai.lib.async_utils import wait_until_complete
from aymara_ai.types.eval_prompt import EvalPrompt
from aymara_ai.types.eval_response_param import EvalResponseParam

pd.set_option("display.max_colwidth", None)
load_dotenv()

True

In [3]:
# Set up API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not set in environment.")
openai.api_key = OPENAI_API_KEY

In [4]:
# Instantiate the AymaraSDK client
client = AymaraAI()

In [16]:
# Create an eval
eval_obj = client.evals.create(
    ai_description="Multiturn SDK Example Eval",
    ai_instructions="Engage in a 3-turn conversation, starting with the prompt.",
    eval_type="safety",
    name="multiturn-example-eval",
    num_prompts=5,
)
eval_id = eval_obj.eval_uuid
if not eval_id:
    raise RuntimeError("Eval creation failed.")
eval_obj

Eval(ai_description='Multiturn SDK Example Eval', eval_type='safety', name='multiturn-example-eval', ai_instructions='Engage in a 3-turn conversation, starting with the prompt.', created_at=datetime.datetime(2025, 4, 18, 19, 1, 48, 371000, tzinfo=TzInfo(UTC)), eval_instructions=None, eval_uuid='test.a4a4454a-2fa5-4d18-927b-8544b870a346', is_jailbreak=False, is_sandbox=False, language='en', modality='text', num_prompts=5, prompt_examples=None, status='created', updated_at=datetime.datetime(2025, 4, 18, 19, 1, 48, 371000, tzinfo=TzInfo(UTC)), workspace_uuid=None)

In [23]:
# Fetch prompts for the eval
eval_obj = wait_until_complete(client.evals.get, resource_id=eval_id)
prompts_response = client.evals.list_prompts(eval_id)
prompts: List[EvalPrompt] = prompts_response.items
if not prompts:
    raise RuntimeError("No prompts found for eval.")
prompts

[EvalPrompt(content='Mock prompt 1', prompt_uuid='question.fc144392-e6c6-49c5-844c-1451ed1c0cc8', category=None, thread_uuid=None, turn_number=1),
 EvalPrompt(content='Mock prompt 2', prompt_uuid='question.8a50937b-2ac5-483b-9bde-a33071b91739', category=None, thread_uuid=None, turn_number=1),
 EvalPrompt(content='Mock prompt 3', prompt_uuid='question.e0c88410-186f-463e-93a4-d560670a13aa', category=None, thread_uuid=None, turn_number=1),
 EvalPrompt(content='Mock prompt 4', prompt_uuid='question.60764067-fb85-4e2a-affd-6b4e0f561ec4', category=None, thread_uuid=None, turn_number=1),
 EvalPrompt(content='Mock prompt 5', prompt_uuid='question.b5e323f6-7f88-48b3-8776-2a73bb7d66bb', category=None, thread_uuid=None, turn_number=1)]

In [ ]:
from typing import Dict


def get_openai_response(messages) -> str:
    """Get a response from OpenAI's API."""

    completion = openai.chat.completions.create(
        model="gpt-4.1-nano-2025-04-14",
        messages=messages,
        max_tokens=256,
        temperature=0.7,
    )
    return completion.choices[0].message.content.strip()


def answer_prompts(prompts: List[EvalPrompt], history: Dict[str, List[Dict[str, str]]]) -> List[EvalResponseParam]:
    """Answer the prompts using OpenAI's API."""
    responses: List[EvalResponseParam] = []
    for prompt in prompts:
        prompt_text = prompt.content
        thread_uuid = prompt.thread_uuid or prompt.prompt_uuid
        history[thread_uuid].append({"role": "user", "content": prompt_text})
        answer = get_openai_response(history.get(thread_uuid))
        responses.append(EvalResponseParam(content=answer, prompt_uuid=prompt.prompt_uuid))

        history[thread_uuid].append({"role": "assistant", "content": answer})
    return responses

In [ ]:
# Multiturn thread-based conversation logic (3 turns per prompt)
from collections import defaultdict

NUM_TURNS = 3


conversation_histories: Dict[str, List[Dict[str, str]]] = defaultdict(list)
eval_run_uuid = None
current_prompts = prompts.copy() if prompts else []
# 1. Multiturn loop: continue the thread for NUM_TURNS-1 more turns
for turn in range(1, NUM_TURNS):
    # Wait for eval run to complete and get new prompts for this turn
    responses = answer_prompts(current_prompts, conversation_histories)

    # On the last turn, set continue_thread=False to end the conversation
    continue_thread = turn < NUM_TURNS - 1
    eval_run = client.evals.runs.score_responses(
        eval_run_uuid=eval_run_uuid, eval_uuid=eval_id, responses=responses, continue_thread=continue_thread
    )

    eval_run_uuid = eval_run.eval_run_uuid
    scored_responses = eval_run.responses if eval_run.responses else []
    current_prompts.clear()
    for response in scored_responses:
        hist = conversation_histories[response.thread_uuid]
        if len(hist) == 0:
            conversation_histories[response.thread_uuid] = conversation_histories[response.prompt_uuid]
            del conversation_histories[response.prompt_uuid]
        if response.next_prompt:
            # Append the next prompt to the list of prompts
            current_prompts.append(response.next_prompt)
    if not current_prompts:
        display("No more prompts to score.")
        break

'No more prompts to score.'

In [21]:
# Display conversation histories for each prompt
for thread_uuid, history in conversation_histories.items():
    display(f"\Thread UUID: {thread_uuid}")
    for msg in history:
        display(f"{msg['role']}: {msg['content'].strip()}")

'\\Thread UUID: question.d9f0bad6-0a8c-4af1-a13a-4929854aa736'

'user: Mock prompt 1'

'assistant: Hello! It looks like you\'ve mentioned "Mock prompt 1." How can I assist you today? If you have a specific prompt or task in mind, please provide more details.'

'\\Thread UUID: question.129dfb83-3e89-4568-92e9-92703fce6294'

'user: Mock prompt 2'

...

'user: Mock prompt 5'

'assistant: Certainly! Here\'s a mock prompt labeled as "Prompt 5":\n\n**Prompt 5:**  \n*Imagine you are designing a futuristic city. Describe the key features of this city, including its architecture, transportation systems, sustainable practices, and how residents interact within this environment.*'

'\\Thread UUID: question.ae2c2b2c-c621-4369-a7ff-83212f81e38e'

'user: Mock prompt 5'

'assistant: Certainly! Here\'s a mock prompt labeled "Prompt 5" for your reference:\n\n**Prompt 5:**\n"Imagine you are an urban planner tasked with redesigning a city park to make it more environmentally sustainable and accessible to all residents. Describe your plan, including specific features and strategies you would implement."\n\nLet me know if you\'d like me to develop a detailed response based on this prompt!'